In [ ]:
%pip install xformers==0.0.19 torch==2.0.0+cu118 torchvision  torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
!sudo apt update
!sudo apt -y install  -qq  ffmpeg  aria2

In [ ]:
#@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /
    
    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
%pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [ ]:
%cd custom_nodes
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux

In [ ]:

%cd comfyui_controlnet_aux
%pip install -r requirements.txt

In [ ]:
%cd /notebooks/ComfyUI

In [ ]:
%cd custom_nodes
!git clone https://github.com/WASasquatch/was-node-suite-comfyui/

In [ ]:
%cd was-node-suite-comfyui
%pip install -r requirements.txt

In [ ]:
%cd /notebooks/ComfyUI
%cd  custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
%pip install -r requirements.txt

In [ ]:
%cd /notebooks/ComfyUI
%cd  custom_nodes
!git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved
%cd ComfyUI-AnimateDiff-Evolved
%pip install -r requirements.txt

In [ ]:
%cd /notebooks/ComfyUI
%cd  custom_nodes
!git clone https://github.com/FizzleDorf/ComfyUI_FizzNodes
%cd ComfyUI_FizzNodes
%pip install -r requirements.txt

In [ ]:
%cd /notebooks/ComfyUI
%cd  custom_nodes
!git clone https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet
%cd ComfyUI-Advanced-ControlNet
%pip install -r requirements.txt


In [ ]:
%cd /notebooks/ComfyUI
%cd  custom_nodes
!rm ControlNet-LLLite-ComfyUI -rf
!git clone https://github.com/kohya-ss/ControlNet-LLLite-ComfyUI.git
%cd ControlNet-LLLite-ComfyUI
%pip install -r requirements.txt

In [ ]:

%cd /notebooks/ComfyUI
%cd  custom_nodes
!git clone https://github.com/storyicon/comfyui_segment_anything.git
%cd comfyui_segment_anything
%pip install -r requirements.txt

In [ ]:
%pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
%cd /notebooks/ComfyUI
%cd  custom_nodes
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite
%cd ComfyUI-VideoHelperSuite
%pip install -r requirements.txt

In [ ]:
%cd /notebooks/ComfyUI

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
%pip install nest-asyncio

In [ ]:
%cd /notebooks/ComfyUI

In [ ]:
!sudo apt-get install lsof

In [ ]:
!kill -9 $(lsof -t -i:8188)


In [ ]:
%autoawait True
import nest_asyncio
nest_asyncio.apply()

%run main.py --preview-method auto  --dont-upcast-attention

In [ ]:
%autoawait True
import nest_asyncio
nest_asyncio.apply()

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel","--region","EU", "--url", "http://0.0.0.0:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

%run main.py